In [1]:
from google.colab import drive
drive.mount('/content/drive')
! pip install dalex

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
  Created wheel for dalex: filename=dalex-1.5.0-py3-none-any.whl size=1043321 sha256=e59e79e4b11e57eca0d50ddc54effdbce010209207d8af1c9381d8670ea9a9e5
  Stored in directory: /root/.cache/pip/wheels/b1/02/58/77ac4cb307fec9f3324c6aa4f9f23a7b0e886e313ebc280257
Successfully built dalex


In [2]:
import sklearn
import dalex as dx
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from dalex.fairness import resample, reweight, roc_pivot

For this homework, train model on the adult income dataset.

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/XAI/adult.csv")
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
column_encoders = {}
for c in df.columns:
  if type(df[c][0]) == str:
    le = preprocessing.LabelEncoder()
    le.fit(df[c])
    df[c] = le.transform(df[c])
    column_encoders[c] = le
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,4,226802,1,7,4,7,3,2,1,0,0,40,39,0
1,38,4,89814,11,9,2,5,0,4,1,0,0,50,39,0
2,28,2,336951,7,12,2,11,0,4,1,0,0,40,39,1
3,44,4,160323,15,10,2,7,0,2,1,7688,0,40,39,1
4,18,0,103497,15,10,4,0,3,4,0,0,0,30,39,0


In [5]:
X, y = df[df.columns[:-1]], df[df.columns[-1]]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

1. Train a model for the selected dataset.

In [7]:
model = xgb.XGBClassifier()

model.fit(X_train, y_train)

XGBClassifier()

2. For the selected protected attribute (age, gender, race) calculate the following fairness coefficients:
- Statistical parity
- Equal opportunity
- Predictive parity

In [8]:
def pf_xgboost_classifier_categorical(model, df):
    df.loc[:, df.dtypes == 'object'] = df.select_dtypes(['object']).apply(lambda x: x.astype('category'))
    return model.predict_proba(df)[:, 1]

explainer = dx.Explainer(model, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)

Preparation of a new explainer is initiated

  -> data              : 16118 rows 14 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 16118 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function pf_xgboost_classifier_categorical at 0x7f20a66cadc0> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000874, mean = 0.239, max = 0.997
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.986, mean = -0.00292, max = 0.994
  -> model_info        : package xgboost

A new explainer has been created!


In [9]:
explainer.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.59495,0.802697,0.683384,0.869959,0.924697


In [10]:
protected_variable = X_test['gender']
privileged_group = column_encoders['gender'].transform(["Male"])[0]

fobject = explainer.model_fairness(protected=protected_variable, privileged=privileged_group)

In [11]:
fobject.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '1'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
       TPR       ACC       PPV       FPR      STP
0  0.80261  1.120669  1.089308  0.132353  0.25974


In [12]:
fobject.plot()

3. Train another model (different hyperparameters, feature transformations etc.) and see how the coefficients *Statistical parity*, *Equal opportunity*, *Predictive parity* behave for it.

In [13]:
X_train_no_gender = X_train.drop(columns=['gender'])
X_test_no_gender = X_test.drop(columns=['gender'])

In [14]:
model_no_gender = xgb.XGBClassifier()

model_no_gender.fit(X_train_no_gender, y_train)

XGBClassifier()

In [15]:
def pf_xgboost_classifier_categorical(model, df):
    df.loc[:, df.dtypes == 'object'] = df.select_dtypes(['object']).apply(lambda x: x.astype('category'))
    return model.predict_proba(df)[:, 1]

explainer_no_gender = dx.Explainer(model_no_gender, X_test_no_gender, y_test, predict_function=pf_xgboost_classifier_categorical)
explainer_no_gender.model_performance()

fobject_no_gender = explainer_no_gender.model_fairness(protected=protected_variable, privileged=privileged_group)

fobject_no_gender.fairness_check()

Preparation of a new explainer is initiated

  -> data              : 16118 rows 13 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 16118 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function pf_xgboost_classifier_categorical at 0x7f20a5d9b160> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.000807, mean = 0.239, max = 0.997
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.986, mean = -0.00276, max = 0.995
  -> model_info        : package xgboost

A new explainer has been created!
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria ex

In [16]:
protected_variable
#privileged_group

7762     0
23881    1
30507    0
28911    1
19484    0
        ..
15968    1
37984    0
31513    1
46024    0
2946     1
Name: gender, Length: 16118, dtype: int64

In [17]:
fobject_no_gender.plot()

4. Apply the selected bias mitigation technique on the first model. Check how *Statistical parity*, *Equal opportunity*, *Predictive parity* coefficients behave after this correction.

In [18]:
# resample
indices_resample = resample(
    protected_variable, 
    y_test, 
    type='preferential', # uniform
    probs=model_no_gender.predict_proba(X_test_no_gender)[:,1], # requires probabilities
    verbose=False
)

model_resample = sklearn.base.clone(model_no_gender)
model_resample.fit(X_train_no_gender.iloc[indices_resample, :], y_train.iloc[indices_resample])
explainer_resample = dx.Explainer(
    model_resample, 
    X_test_no_gender, 
    y_test, 
    label='XGBClassifier with Resample mitigation',
    verbose=False
)
fobject_resample = explainer_resample.model_fairness(
    protected_variable, 
    privileged_group
)

5. Compare the quality (performance) of the three models with their fairness coefficients. Is there any correlation?

In [19]:
fobject_no_gender.plot([fobject_resample], show=False)